#  Seting environment an loading

In [1]:
%load_ext autoreload
%autoreload 2
%run ../nb_config.py 

running notebook configuration


In [2]:
path=os.environ.get('LOCAL_PATH')

In [3]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf 
from sklearn.metrics import mean_squared_error, accuracy_score

from src.data_api import sources
from src import utils

 
import pandas as pd
import numpy as np 
import random as rd

import seaborn as sns
  
  

In [4]:
import dash
from dash import html
from dash import dcc
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import plotly.express as px
from dash.dependencies import Input, Output
from datetime import time , timedelta, date ,datetime

In [5]:
conf = utils.get_conf()

In [6]:
wiki_prices=pd.read_pickle(path+"data/processed/wiki_prices.pkl")

In [7]:
wiki_prices=wiki_prices[[  'adj_close',
       'adj_volume', 'sector', 'target', 'x_returns5d', 'x_diff5d',
       'diff_target', 'x_returns1d', 'x_returns1y', 'x_diff1y',
       'x_returns_1y_sdmean', 'x_returns_1y_zscr', 'x_returns_1y_tsmooth',
      ]]

In [8]:
x_cols_to_visualize= [  'adj_close',
       'adj_volume', 'sector',   'x_returns5d', 'x_diff5d',
       'diff_target', 'x_returns1d', 'x_returns1y', 'x_diff1y',
       'x_returns_1y_sdmean', 'x_returns_1y_zscr', 'x_returns_1y_tsmooth',
      ]

In [9]:
wiki_prices=wiki_prices[wiki_prices.index.isin(["AAPL","AMZN","GOOG","FB"],level="ticker")]

In [10]:
tickers=pd.unique(wiki_prices.index.get_level_values("ticker"))


# Dash parameters

In [11]:
# the style arguments for the sidebar.
SIDEBAR_STYLE = {
    'position': 'fixed',
    'top': 0,
    'left': 0,
    'bottom': 0,
    'width': '20%',
    'padding': '20px 10px',
    'background-color': '#f8f9fa'
}

# the style arguments for the main content page.
CONTENT_STYLE = {
    'margin-left': '25%',
    'margin-right': '5%',
    'top': 0,
    'padding': '20px 10px'
}

TEXT_STYLE = {
    'textAlign': 'center',
    'color': '#191970'
}

CARD_TEXT_STYLE = {
    'textAlign': 'center',
    'color': '#0074D9'
}

In [12]:
controls = html.Div(  
    [
        html.P('Ticker', style={
            'textAlign': 'center'
        }),
        dcc.Dropdown(
            id='dropdown_ticker',
            options=[{'label': ticker, 'value': ticker} for ticker in tickers],
            value=['AMZN', 'AAPL'],
            multi=True 
        ),
        html.Br(),
        html.P('Feat', style={
            'textAlign': 'center'
        }),
        dcc.Dropdown(
            id='dropdown_feat',
            options=[{'label': feat, 'value': feat} for feat in wiki_prices.columns],
            value='adj_close',
            multi=False 
        ),
        html.Br(),

        html.P('Range Slider', style={
            'textAlign': 'center'
        }),
        dcc.RangeSlider(
            id='range_slider',
            min=0,
            max=20,
            step=0.5,
            value=[5, 15]
        ),
        html.P('Check Box', style={
            'textAlign': 'center'
        }),
        html.Br(),
        html.P('Radio Items', style={
            'textAlign': 'center'
        }),

    ]
)

In [13]:
sidebar = html.Div(
    [
        html.H2('Parameters', style=TEXT_STYLE),
        html.Hr(),
        controls
    ],
    style=SIDEBAR_STYLE,
)

In [14]:
content_first_row = dbc.Row( html.Div( dcc.Graph(id = 'simple_feat_plot' )))

In [15]:
content = html.Div(
    [
        html.H2('Analytics Dashboard Template', style=TEXT_STYLE),
        html.Hr(),
        content_first_row,
        #content_second_row,
        #content_third_row,
        #content_fourth_row
    ],
    style=CONTENT_STYLE
)

In [16]:
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = html.Div([sidebar, content])

In [17]:
#callbacks
@app.callback(
    Output("simple_feat_plot", "figure"), 
    [Input("dropdown_ticker", "value") ,   Input("dropdown_feat", "value")])
    
def simple_feat_plot( tickers,feat    ):
    df=wiki_prices.copy()

    fig = go.Figure()
    for ticker in tickers:
        fig.add_trace(go.Scatter(
        name=ticker,
        mode="lines", x=df.index.get_level_values("date"), y=df.loc[ticker,feat],
    ))

    fig.update_xaxes(showgrid=True, ticklabelmode="period")
    fig.update_layout(xaxis_title="date", yaxis_title=feat)
    return fig


##### me quede por aquí 

In [ ]:
#calling the app
if __name__ == '__main__': 
    app.run_server(host='127.0.0.1', port='8050', debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Nov/2021 17:02:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2021 17:02:14] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_0_0m1632235559.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2021 17:02:14] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_0_0m1632235559.min.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2021 17:02:14] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v1_0_1-rc1m1635501592.min.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2021 17:02:14] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_0_0m1632235559.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2021 17:02:14] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_0_0m1632235559.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2021 17:02:14] "GET /_dash-component-suites/dash/deps/react@16.v2_0_0m1632235559.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/202